In [1]:
import pandas as pd

In [2]:
df =pd.read_csv("../data/datasets_13720_18513_insurance.csv")

In [3]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [4]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [9]:
df.columns


Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [10]:
numeric_features=[]
catigorical_features=[]
for col in df.columns[:-1]:
    if df[col].dtype == 'O':
        catigorical_features.append(col)
    else:
        numeric_features.append(col)

In [11]:
catigorical_features

['sex', 'smoker', 'region']

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [13]:
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
catigorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='if_binary'))])

In [14]:
colum_transformer = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numeric_features),
        ('cat', catigorical_transformer, catigorical_features)])

In [15]:
reg = Pipeline(steps=[('columnTransform', colum_transformer),
                      ('regression', LinearRegression())])

In [16]:
reg.fit(df.iloc[:,:-1],df.iloc[:,-1])

Pipeline(steps=[('columnTransform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('regression', LinearRegression())])

In [17]:
reg.named_steps['regression'].coef_

array([ 3607.47273619,  2067.69196584,   572.99820995,  -131.3143594 ,
       23848.53454191,   587.00923503,   234.0453356 ,  -448.01281436,
        -373.04175627])

In [18]:
dataCols = list(df.columns[:-1])

## export files & test load

In [19]:
import pickle

In [21]:
pickle.dump(reg, open('../data/pipline.pickle', 'wb'))
pickle.dump(dataCols, open('../data/columnNames.pickle', 'wb'))

In [38]:
reg2 = pickle.load(open('../data/pipline.pickle', 'rb'))
dataCols = pickle.load(open('../data/columnNames.pickle', 'rb'))


In [25]:
reg2.named_steps['regression'].coef_

array([ 3607.47273619,  2067.69196584,   572.99820995,  -131.3143594 ,
       23848.53454191,   587.00923503,   234.0453356 ,  -448.01281436,
        -373.04175627])

## test prediction

In [26]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [27]:
import numpy as np
test = pd.DataFrame(np.array([[28,'male',30,0,'no','northwest']]),columns=dataCols)
test

,age,sex,bmi,children,smoker,region
0,28,male,30,0,no,northwest


In [28]:
list(reg2.predict(test))

[4944.96464438397]

## test hitting url

In [31]:
import requests
import json

In [36]:
url = 'http://0.0.0.0:5000/predict'
#url="https://insurance-demo.azurewebsites.net/predict"
j_data = json.dumps([dict(zip(dataCols,[28,'female',30.0,0,'no','northwest']))])
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

r = requests.post(url, data=j_data, headers=headers)
print(r, r.text)

<Response [200]> {"prediction":[5076.27900377908]}



## other testing

In [37]:
pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.
